# Developing multiple-shooting in Sun-B1 dynamics

2022.11.23

In [1]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
# import joptimise
using Printf
using JSON


In [2]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
# using R3BP
include("../src/SailorMoon.jl")   # relative path to main file of module
param3b = SailorMoon.dynamics_parameters()


┌ Warning: Replacing docs for `Main.SailorMoon.integrate_rk4 :: Union{Tuple{ODEProblem, Real}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool, Bool}}` in module `Main.SailorMoon`
└ @ Base.Docs docs\Docs.jl:240


Main.SailorMoon.dynamics_params(0.987849414390376, 0.01215058560962404, 328900.5598102475, 384748.32292972936, 375700.3437894195, 388.8212386592645, -0.9251999994040079, 0.9251999994040079, 0.07480000059599223, 0.01709689063726318, 7.601281331451572)

In [3]:
plotly()

┌ Warning: For saving to png with the `Plotly` backend `PlotlyBase` and `PlotlyKaleido` need to be installed.
│   err =
│    ArgumentError: Package PlotlyBase not found in current path:
│    - Run `import Pkg; Pkg.add("PlotlyBase")` to install the PlotlyBase package.
│    
└ @ Plots C:\Users\yujit\.julia\packages\Plots\nuwp4\src\backends.jl:545


Plots.PlotlyBackend()

In [4]:
function plot_circle(radius, x, y, n=100)
    circle = zeros(2,n)
    thetas = LinRange(0.0, 2π, n)
    for i = 1:n
        circle[1,i] = radius*cos(thetas[i]) + x
        circle[2,i] = radius*sin(thetas[i]) + y
    end
    return circle
end

plot_circle (generic function with 2 methods)

In [5]:
moon = plot_circle(1-param3b.mu2, param3b.as , 0.0)
earth = plot_circle(param3b.mu2, param3b.as, 0.0)
earth_ub = plot_circle(param3b.mu2+6375/param3b.lstar, param3b.as, 0.0)
earth_lb = plot_circle(param3b.mu2-6375/param3b.lstar, param3b.as, 0.0)

moon_soi_outer = plot_circle(1-param3b.mu2+66000/param3b.lstar, param3b.as, 0.0)
println(param3b.mu2 - 6375/param3b.lstar)

-0.004418687393196386


In [6]:
# solver settings within fitness function
# https://diffeq.sciml.ai/stable/solvers/dynamical_solve/#Symplectic-Integrators
#method = RK4()  # CalvoSanz4()
#reltol = 1e-10
#abstol = 1e-10
dt = 0.001

param3b = SailorMoon.dynamics_parameters()
lps = SailorMoon.lagrange_points(param3b.mu2)

lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")

x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1);

# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, Tsit5(), 1e-12, 1e-12, 0.005
);

Halo guess Az_km: 1200.0
Unrecognized keyword arguments: 

┌ Warning: Unrecognized keyword arguments found. Future versions will error.
│ The only allowed keyword arguments to `solve` are:
│ (:dense, :saveat, :save_idxs, :tstops, :tspan, :d_discontinuities, :save_everystep, :save_on, :save_start, :save_end, :initialize_save, :adaptive, :abstol, :reltol, :dt, :dtmax, :dtmin, :force_dtmin, :internalnorm, :controller, :gamma, :beta1, :beta2, :qmax, :qmin, :qsteady_min, :qsteady_max, :qoldinit, :failfactor, :calck, :alias_u0, :maxiters, :callback, :isoutofdomain, :unstable_check, :verbose, :merge_callbacks, :progress, :progress_steps, :progress_name, :progress_message, :timeseries_errors, :dense_errors, :weak_timeseries_errors, :weak_dense_errors, :wrap, :calculate_error, :initializealg, :alg, :save_noise, :delta, :seed, :alg_hints, :kwargshandle, :trajectories, :batch_size, :sensealg, :advance_to_tstop, :stop_at_next_tstop, :default_set, :second_time, :prob_choice, :alias_jump, :alias_noise)
│ 
│ See https://diffeq.sciml.ai/stable/basics/common_s

[:method]

Unrecognized keyword arguments: [:method]

Unrecognized keyword arguments: [:method]

Unrecognized keyword arguments: [:method]

Unrecognized keyword arguments: [:method]

Unrecognized keyword arguments: [:method]



┌ Warning: Unrecognized keyword arguments found. Future versions will error.
│ The only allowed keyword arguments to `solve` are:
│ (:dense, :saveat, :save_idxs, :tstops, :tspan, :d_discontinuities, :save_everystep, :save_on, :save_start, :save_end, :initialize_save, :adaptive, :abstol, :reltol, :dt, :dtmax, :dtmin, :force_dtmin, :internalnorm, :controller, :gamma, :beta1, :beta2, :qmax, :qmin, :qsteady_min, :qsteady_max, :qoldinit, :failfactor, :calck, :alias_u0, :maxiters, :callback, :isoutofdomain, :unstable_check, :verbose, :merge_callbacks, :progress, :progress_steps, :progress_name, :progress_message, :timeseries_errors, :dense_errors, :weak_timeseries_errors, :weak_dense_errors, :wrap, :calculate_error, :initializealg, :alg, :save_noise, :delta, :seed, :alg_hints, :kwargshandle, :trajectories, :batch_size, :sensealg, :advance_to_tstop, :stop_at_next_tstop, :default_set, :second_time, :prob_choice, :alias_jump, :alias_noise)
│ 
│ See https://diffeq.sciml.ai/stable/basics/common_s

Linear stability ν = 618.7618470972243


In [7]:
tmax_si = 280e-3 * 4   # N
isp_si = 1800   # sec
mdot_si = tmax_si / (isp_si * 9.81)
mstar = 2500  # kg
rp_parking = (6378+200)/param3b.lstar   # parking orbit radius

tmax = AstrodynamicsBase.dimensional2canonical_thrust(
    tmax_si, mstar, param3b.lstar, param3b.tstar
)
mdot = AstrodynamicsBase.dimensional2canonical_mdot(
    mdot_si, mstar, param3b.tstar
)

0.009531869635160265

In [ ]:
# put parameters here

θsf = 3.141592654
ϕ0 = 0.0

# apoapsis state
sv_mid = [
    387.5004415234835, 
    4.0253946191092815, 
    6.789181649232e-23, 
    0.10076717389510526, 
    0.03283640579320107, 
    2.4278753237168335e-23, 
    1.1328266033657977
]

tof = 23.32911501
tof_leo2mid = tof - 13.935  # be careful, this is dt2, not dt1

# periapsis parameters
rp = 0.027835119
ra = 5.357114163
α = -0.703712317

# make this variable as well
rp_parking = rp 

In [35]:
## another inputs
θsf = 3.351032164
ϕ0 = 2.722713633

# apoapsis state
sv_mid = [387.6962457, 4.46534471, 3.62E-20, 0.130905689, 0.026472285, 3.91E-22,1.0]

tof = 24.71281274
tof_leo2mid = 10.39102352  # be careful, this is dt2, not dt1

# periapsis parameters
rp = 0.017102188
ra = 5.328870741
α = -0.74853434

# make this variable as well
rp_parking = rp 
ωM = param3b.oml
ωb = param3b.omb
θm0 = θmf + ωM * (-tof)

-23.073733842729563

In [12]:
param3b.lstar*0.017102188- 6378

202.0381514289429

In [37]:
t_sidereal = 27.3217   * 86400      # sec
t_synodic  = 29.530588 * 86400      # sec
θmf = pi - θsf  # placeholder
ωM = param3b.oml
ωb = param3b.omb
θm0 = θmf + ωM * (-tof)

params = [
    param3b.mu2, param3b.mus, param3b.as, θmf, param3b.oml, param3b.omb, 0.0, 0.0, 0.0, 0.0, 0.0,
    SailorMoon.dv_sun_dir_angles
]    

# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO2(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, 1e-6, Tsit5(), 1e-12, 1e-12, 0.005
);

xf = SailorMoon.set_terminal_state2(ϕ0, θmf, param3b, LPOArrival)
# in Sun-B1 frame
u0_test = vcat(SailorMoon.transform_EMrot_to_SunB1(xf, θsf, param3b.oms, param3b.as), 1.0)
println(u0_test)

_prob_base = ODEProblem(SailorMoon.rhs_bcr4bp_sb1frame2!, u0_test, [0, -tof], params);
print(params)

[387.68087182133075, -0.1599522821984354, -1.5537332697416227e-21, -0.09612970299421639, 1.0984974147236626, 5.452589045975519e-21, 1.0]
Any[0.01215058560962404, 328900.5598102475, 388.8212386592645, -0.2094395104102067, 0.9251999994040079, 0.07480000059599223, 0.0, 0.0, 0.0, 0.0, 0.0, Main.SailorMoon.dv_sun_dir_angles]

In [ ]:
sol_ = SailorMoon.integrate_rk4(_prob_base, 0.001);
print(sol_.u[end])

In [ ]:
pcart = plot(hcat(sol_.u...)[1,:], hcat(sol_.u...)[2,:], label="backward(from LLO):0.001", set_aspect=:equal, aspect_ratio=:equal, size=(700,700))

In [ ]:
pcart = plot(hcat(sol_.u...)[1,:], hcat(sol_.u...)[2,:], label="backward(from LLO):0.001", set_aspect=:equal, size=(700,700))
plot!(pcart, earth[1,:], earth[2,:], c=:green, lw=1.0, label="earth")
plot!(pcart, moon[1,:], moon[2,:], c=:orange, lw=1.0, label="moon")
plot!(pcart, moon_soi_outer[1,:], moon_soi_outer[2,:], c=:black, lw=1.0, label="moon_soi_outer")

u_midpoint = sol_.u[15000]
scatter!(pcart, [u_midpoint[1]], [u_midpoint[2]], marker=:circle)

pcart

In [ ]:
u_midpoint

### Check the time-reversibility of integral 

In [ ]:
### haven't figured out why, but making the velocity, oml, and omb negative works somehow...

θm0 = θmf + ωM * (-tof)
params = [
    param3b.mu2, param3b.mus, param3b.as, θm0, -param3b.oml, -param3b.omb, 0.0, 0.0, 0.0, 0.0, 0.0,
    SailorMoon.dv_no_thrust
]   
println(params)

# u0_test = [
#     388.83725006664025, 
#     -0.008060103011816226, 
#     1.7345154878917807e-24, 
#     -1.4287345247061145, 
#     10.131022000599954, 
#     6.151029489849641e-21, 
#     1.2476475049907993
# ]

state_leo = vcat(SailorMoon.paramIni_to_sb1(rp, α, ra, θm0, param3b.oml, param3b.mu2, param3b.as), 1.0)
state_leo[4:6] = - state_leo[4:6]

_prob = remake(_prob_base, u0=state_leo, t=[0, tof], p=params)

sol_2 = SailorMoon.integrate_rk4(_prob, 0.001);

In [ ]:
plot!(pcart, hcat(sol_2.u...)[1,:], hcat(sol_2.u...)[2,:], color=:green, aspect_ratio=:equal, label="forward(from LEO): 0.001")

In [ ]:
sol_tsit5 = solve(_prob, Tsit5(), reltol=1e-12, abstol=1e-12)
plot!(pcart, Array(sol_tsit5)[1,:], Array(sol_tsit5)[2,:], color=:red, linewidth=1.0, label="sol", aspect_ratio=:equal, linestyle=:solid)


### Develop propagation function

In [15]:
n_arc = 5

5

In [16]:
# ballistic time right after launch
ballistic_time = 1*86400 / param3b.tstar
ballistic_time_back = 1*86400 / param3b.tstar

dt = 0.001

0.001

In [17]:
function unpack_x(x::AbstractVector{T}, verbose::Bool=true) where T
    # unpack
    nx = length(x)
    x_LEO = x[1:5+3n_arc]
    x_mid = x[6+3n_arc:14+9n_arc]    # x[5+3n_arc:4+3n_arc+9+6n_arc]
    x_LPO = x[15+9n_arc:18+12n_arc]  # x[14+9n_arc:13+9n_arc+4+3n_arc]
    
    # get time of flights
    tofs = [x_LEO[5], x_mid[8], x_mid[9], x_LPO[4]]
    θf = x_LPO[1]
    θs = [
        θf - param3b.oms*sum(broadcast(abs, tofs)),
        θf - param3b.oms*sum(broadcast(abs, tofs[3:4])),
        θf
    ]
    # print message
    if verbose
        @printf("ToF per arc  : %3.3f, %3.3f, %3.3f, %3.3f\n", tofs...)
        @printf("Phase angles : %3.3f, %3.3f, %3.3f\n", θs...)
        println("θf: ", θf)
    end
    return x_LEO, x_mid, x_LPO, tofs, θs
end

unpack_x (generic function with 2 methods)

In [18]:
function get_LEO_state(x_LEO, θs, verbose::Bool=false) 
    rp_input, ra_input, α = x_LEO[1:3]
    sv0_sunb1 = vcat(
        SailorMoon.paramIni_to_sb1(rp, α, ra, θm0, param3b.oml, param3b.mu2, param3b.as), 
        x_LEO[4]
        )
    
    if verbose
        println("SMA [km]: ", (rp_input + ra_input)*param3b.lstar)
        println("ra  [km]: ", x_LEO[1]*param3b.lstar)
        println("Energy in inertial frame: ", -(param3b.mu2)/(rp_input + ra_input))
        println("sv0_sb1: ", (sv0_sunb1[1:3]-[param3b.as,0,0])*param3b.lstar, sv0_sunb1[4:6]*param3b.lstar/param3b.tstar)
    end
    

    # ballistic propagation with small time-steps
    params = [
        param3b.mu2, param3b.mus, param3b.as, pi-θs[1], ωM, ωb, 0.0, 0.0, 0.0, mdot, tmax,
        SailorMoon.dv_sun_dir_angles
    ]
    
    _prob = remake(_prob_base; tspan=[0,ballistic_time], u0 = sv0_sunb1, p = params)
    sol_ballistic_fwd = SailorMoon.integrate_rk4(_prob, 0.001);
    θ_iter = θs[1] + param3b.oms*sol_ballistic_fwd.t[end]

    
    return sol_ballistic_fwd.u[end], θ_iter, sol_ballistic_fwd
end

get_LEO_state (generic function with 2 methods)

In [19]:
function get_LPO_state(x_LPO, θs, verbose::Bool=false)
    xf = SailorMoon.set_terminal_state2(x_LPO[2], θs[3], param3b, LPOArrival)

    # transform to Sun-B1 frame
    svf_sunb1 = vcat(
        SailorMoon.transform_EMrot_to_SunB1(xf, θs[3], param3b.oms, param3b.as),
        x_LPO[3]   # mf
    )    
        
    # ballistic propagation with small time-steps
    params = [
        param3b.mu2, param3b.mus, param3b.as, pi - θs[3], ωM, ωb, 0.0, 0.0, 0.0, mdot, tmax,
        SailorMoon.dv_no_thrust
    ]
    println("params: ", params)
    _prob = remake(
        _prob_base; tspan=[0,-ballistic_time_back], 
        u0 = svf_sunb1, p = params
    )
    sol_ballistic_bck = SailorMoon.integrate_rk4(_prob, 0.001);
    θ_iter = θs[3] - param3b.oms*ballistic_time_back
    
    if verbose
        println("svf_sunb1: \n", svf_sunb1)
    end
    return sol_ballistic_bck.u[end], θ_iter, sol_ballistic_bck
end

get_LPO_state (generic function with 2 methods)

In [20]:
function propagate_arc!(sv0, θ0, tspan, x_control, get_sols::Bool, sol_param_list, name::String)
    sv_iter = [el for el in sv0]
    θ_iter = 1*θ0
    for i = 1:n_arc
        τ, γ, β = x_control[1+3*(i-1) : 3*i]
        
        params = [
            param3b.mu2, param3b.mus, param3b.as, pi - θ_iter, ωM, ωb, τ, γ, β, mdot, tmax,
            SailorMoon.dv_sun_dir_angles
        ]
        _prob = remake(_prob_base; tspan=tspan, u0 = sv_iter, p = params)
        sol = SailorMoon.integrate_rk4(_prob, dt);
        #sol = DifferentialEquations.solve(_prob, RK4(), reltol=1e-10, abstol=1e-10)
        if get_sols
            push!(sol_param_list, [sol, params, name])
        end
        # update θ0 and sv0
        θ_iter += param3b.oms*sol.t[end]
        sv_iter = sol.u[end]
    end
    return sv_iter
end

propagate_arc! (generic function with 1 method)

In [23]:
multishoot_trajectory = function (x::AbstractVector{T}, get_sols::Bool=false, verbose::Bool=false) where T
    # unpack decision vector
    x_LEO, x_mid, x_LPO, tofs, θs = unpack_x(x)
    
    # initialize storage
    sol_param_list = []
    
    # propagate from LEO forward
    sv0_LEO, θ0_leo, sol_ballistic_fwd = get_LEO_state(x_LEO, θs, verbose)
    svf_LEO = propagate_arc!(
        sv0_LEO, θ0_leo, [0, tofs[1]/n_arc], x_LEO[5 : end],
        get_sols, sol_param_list, "leo_arc"
    )
    
    # propagate midpoint backward
    svm0 = x_mid[1:7]  # state-vector at midpoint, [r,v,mass]
    svf_mid_bck = propagate_arc!(
        svm0, θs[2], [0, -tofs[2]/n_arc], x_mid[10 : end],
        get_sols, sol_param_list, "mid_bck_arc"
    )
    
    # propaagte midpoint forward
    svf_mid_fwd = propagate_arc!(
        svm0, θs[2], [0, tofs[3]/n_arc], x_mid[10+3n_arc : end],
        get_sols, sol_param_list, "mid_fwd_arc"
    )
    
    # propagate from LPO backward
    sv0_LPO, θ0_lpo, sol_ballistic_bck = get_LPO_state(x_LPO, θs, verbose)
    svf_lpo = propagate_arc!(
        sol_ballistic_bck.u[end], θ0_lpo, [0, -tofs[4]/n_arc], x_LPO[5 : end],
        get_sols, sol_param_list, "lpo_arc"
    )
    
    # residuals
    res = vcat(svf_mid_bck - svf_LEO, svf_lpo - svf_mid_fwd)[:]

    # output
    if get_sols == false
        return res
    else
        return res, sol_param_list, [sol_ballistic_fwd,sol_ballistic_bck], tofs
    end
end

#7 (generic function with 3 methods)

In [26]:
tof_mid2lpo = tof - tof_leo2mid

# create test decision vector
τ_ig = 0.0
# x_LEO = [ra, rp, α, m0, tof, controls...]
ig_x_LEO = vcat(
    [rp, ra,  α, 1.0, tof_leo2mid/2],
    vcat([[τ_ig,0,0] for i = 1:n_arc]...)
);
ig_x_mid = vcat(
    sv_mid, tof_leo2mid/2, tof_mid2lpo/2, 
    vcat([[τ_ig,0,0] for i = 1:2n_arc]...)
);
# x_LPO = [θf, ϕ, mf, tof, controls...]
ig_x_LPO = vcat(
    [θsf, ϕ0, 1.0, tof_mid2lpo/2],
    vcat([[τ_ig,0,0] for i = 1:n_arc]...)
);
ig_x = vcat(ig_x_LEO, ig_x_mid, ig_x_LPO);
x_LEO, x_mid, x_LPO, tofs, θs = unpack_x(ig_x, true);

ToF per arc  : 5.196, 5.196, 7.161, 7.161
Phase angles : 26.215, 16.602, 3.351
θf: 3.351032164


In [38]:
@time res, sol_param_list, sols_ballistic, _ = multishoot_trajectory(ig_x, true, true);
println(res)
norm(res)

ToF per arc  : 5.196, 5.196, 7.161, 7.161
Phase angles : 26.215, 16.602, 3.351
θf: 3.351032164
SMA [km]: 2.0568541188604832e6
ra  [km]: 6580.038151428943
Energy in inertial frame: -0.002272848323587918
sv0_sb1: [7013.863861343279, -8606.910329513894, 0.0][7.47900628773787, 8.051790314399982, 0.0]
params: Any[0.01215058560962404, 328900.5598102475, 388.8212386592645, -0.2094395104102067, 0.9251999994040079, 0.07480000059599223, 0.0, 0.0, 0.0, 0.009531869635160265, 0.16435558384609777, Main.SailorMoon.dv_no_thrust]
svf_sunb1: 
[387.68087182133075, -0.1599522821984354, -1.5537332697416227e-21, -0.09612970299421639, 1.0984974147236626, 5.452589045975519e-21, 1.0]
  1.168517 seconds (13.82 M allocations: 351.445 MiB, 9.90% gc time, 61.62% compilation time)
[-0.1828047538123201, 0.7679557007678324, 2.6406553789730615e-20, -0.03838812232839925, 0.26659413715951374, 3.418776125126439e-21, 0.0, 0.7840061324841372, -1.4506311810092245, -1.523854673230642e-20, 0.9734326266158879, 0.59261580687693

2.171055188260212

In [39]:
arcs_color = Dict(
    "leo_arc" => :navy, 
    "mid_bck_arc" => :red1, 
    "mid_fwd_arc" => :darkorange, 
    "lpo_arc" => :green
)

pcart = plot(
    size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4, legend=false
)
plot!(pcart, earth[1,:], earth[2,:], c=:green, lw=1.0, label="earth")
plot!(pcart, earth_ub[1,:], earth_ub[2,:], c=:black, lw=1.0, label="earth")
plot!(pcart, earth_lb[1,:], earth_lb[2,:], c=:black, lw=1.0, label="earth")

plot!(pcart, moon[1,:], moon[2,:], c=:orange, lw=1.0, label="moon")
plot!(pcart, moon_soi_outer[1,:], moon_soi_outer[2,:], c=:black, lw=1.0, label="moon_soi_outer")

# plot!(hcat(sol_.u...)[1,:], hcat(sol_.u...)[2,:], color=:black, label="backward(from LLO):0.001", set_aspect=:equal, size=(700,700))


# ballistic legs
for sol_ballistic in sols_ballistic
    plot!(pcart, hcat(sol_ballistic.u...)[1,:], hcat(sol_ballistic.u...)[2,:], c=:dodgerblue, label=false)
end

# trajectory
for i = 1:length(sol_param_list)
    sol, _, name = sol_param_list[i]
    #if name == "mid_bck_arc" || name == "mid_fwd_arc" || name == "lpo_arc"
    plot!(pcart, SailorMoon.Array(sol)[1,:], SailorMoon.Array(sol)[2,:], 
        linewidth=1.5, label="$name", c=arcs_color[name])
    #end
end
# control node
scatter!(pcart, [x_mid[1]], [x_mid[2]], marker=:circle, color=:black, label="MP")
scatter!(pcart, [param3b.as], [0.0], marker=:circle, color=:black)

# plot the earth when the initial guess is terminated 
θm0 = π - (θsf - param3b.oms * tof)
println(θm0)
θm02 = π - θsf + param3b.oml * (-tof)
println(θm02)
e0x = param3b.as - param3b.mu2*cos(θm0)
e0y = param3b.mu2*sin(θm0)
earth0 = plot_circle(6375/param3b.lstar, e0x, e0y)

plot!(pcart, earth0[1,:], earth0[2,:], c=:red, lw=1.0, label="earth")
scatter!(pcart, [e0x], [e0y], color=:red, markersize=2.5)


pcart

-23.073733842729563
-23.073733842729563


In [ ]:
param3b.lstar * 0.0204374601170627 - 6375

In [ ]:
param3b.lstar